In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext

In [2]:
sc = SparkContext()

In [3]:
log_file_path = '/home/filippo/PycharmProjects/httpredictamancy/resources/UofS_access_log'

In [4]:
sqlContext = SQLContext(sc)

# Fix the columns with null content

In [5]:
base_df = sqlContext.read.text(log_file_path)

In [6]:
base_df.printSchema()

root
 |-- value: string (nullable = true)



In [7]:
base_df.show(truncate=False)

+-------------------------------------------------------------------------------------------------------+
|value                                                                                                  |
+-------------------------------------------------------------------------------------------------------+
|202.32.92.47 - - [01/Jun/1995:00:00:59 -0600] "GET /~scottp/publish.html" 200 271                      |
|ix-or7-27.ix.netcom.com - - [01/Jun/1995:00:02:51 -0600] "GET /~ladd/ostriches.html" 200 205908        |
|ram0.huji.ac.il - - [01/Jun/1995:00:05:44 -0600] "GET /~scottp/publish.html" 200 271                   |
|eagle40.sasknet.sk.ca - - [01/Jun/1995:00:08:06 -0600] "GET /~lowey/" 200 1116                         |
|eagle40.sasknet.sk.ca - - [01/Jun/1995:00:08:19 -0600] "GET /~lowey/kevin.gif" 200 49649               |
|cdc8g5.cdc.polimi.it - - [01/Jun/1995:00:11:03 -0600] "GET /~friesend/tolkien/rootpage.html" 200 461   |
|freenet2.carleton.ca - - [01/Jun/1995:00:16:5

In [8]:
from pyspark.sql.functions import split, regexp_extract

In [9]:
split_df = base_df.select(regexp_extract('value', r'^([^\s]+\s)', 1).alias('host'),
                          regexp_extract('value', r'^.*\[(\d\d/\w{3}/\d{4}:\d{2}:\d{2}:\d{2} -\d{4})]', 1).alias('timestamp'),
                          regexp_extract('value', r'^.*"\w+\s+([^\s]+)\s+HTTP.*"', 1).alias('path'),
                          regexp_extract('value', r'^.*"\s+([^\s]+)', 1).cast('integer').alias('status'),
                          regexp_extract('value', r'^.*\s+(\d+)$', 1).cast('integer').alias('content_size'))
split_df.show(truncate=False)

+------------------------+--------------------------+----+------+------------+
|host                    |timestamp                 |path|status|content_size|
+------------------------+--------------------------+----+------+------------+
|202.32.92.47            |01/Jun/1995:00:00:59 -0600|    |200   |271         |
|ix-or7-27.ix.netcom.com |01/Jun/1995:00:02:51 -0600|    |200   |205908      |
|ram0.huji.ac.il         |01/Jun/1995:00:05:44 -0600|    |200   |271         |
|eagle40.sasknet.sk.ca   |01/Jun/1995:00:08:06 -0600|    |200   |1116        |
|eagle40.sasknet.sk.ca   |01/Jun/1995:00:08:19 -0600|    |200   |49649       |
|cdc8g5.cdc.polimi.it    |01/Jun/1995:00:11:03 -0600|    |200   |461         |
|freenet2.carleton.ca    |01/Jun/1995:00:16:54 -0600|    |200   |5759        |
|red.weeg.uiowa.edu      |01/Jun/1995:00:18:14 -0600|    |200   |461         |
|interchg.ubc.ca         |01/Jun/1995:00:23:53 -0600|    |200   |2460        |
|interchg.ubc.ca         |01/Jun/1995:00:24:17 -0600

In [10]:
base_df.filter(base_df['value'].isNull()).show()

+-----+
|value|
+-----+
+-----+



In [11]:
bad_rows_df = split_df.filter(split_df['host'].isNull() |
                              split_df['timestamp'].isNull() |
                              split_df['path'].isNull() |
                              split_df['status'].isNull() |
                             split_df['content_size'].isNull())


In [12]:
bad_rows_df.show()

+--------------------+--------------------+--------------------+------+------------+
|                host|           timestamp|                path|status|content_size|
+--------------------+--------------------+--------------------+------+------------+
|    edu219.usask.ca |01/Jun/1995:09:24...|          /education|   302|        null|
|    edu219.usask.ca |01/Jun/1995:09:24...|          /education|   302|        null|
|    edu219.usask.ca |01/Jun/1995:09:24...|          /education|   302|        null|
|    edu267.usask.ca |01/Jun/1995:09:30...|          /education|   302|        null|
|gatekeeper.mitre....|01/Jun/1995:09:34...|/dcs/courses/cai/...|   302|        null|
|    edu269.usask.ca |01/Jun/1995:09:34...|          /education|   302|        null|
|    edu262.usask.ca |01/Jun/1995:09:43...|          /education|   302|        null|
|    edu219.usask.ca |01/Jun/1995:09:47...|          /education|   302|        null|
|   jjh1.lib.msu.edu |01/Jun/1995:10:09...|            /library| 

In [13]:
from pyspark.sql.functions import col, sum

In [14]:
def count_null(col_name):
  return sum(col(col_name).isNull().cast('integer')).alias(col_name)

# Build up a list of column expressions, one per column.
#
# This could be done in one line with a Python list comprehension, but we're keeping
# it simple for those who don't know Python very well.
exprs = []
for col_name in split_df.columns:
  exprs.append(count_null(col_name))

# Run the aggregation. The *exprs converts the list of expressions into
# variable function arguments.
split_df.agg(*exprs).show()

+----+---------+----+------+------------+
|host|timestamp|path|status|content_size|
+----+---------+----+------+------------+
|   0|        0|   0|     0|       46289|
+----+---------+----+------+------------+



# Fix the columns with null content

In [15]:
# Replace all null content_size values with 0.
cleaned_df = split_df.na.fill({'content_size': 0})

In [16]:
# Ensure that there are no nulls left.
exprs = []
for col_name in cleaned_df.columns:
  exprs.append(count_null(col_name))

cleaned_df.agg(*exprs).show()

+----+---------+----+------+------------+
|host|timestamp|path|status|content_size|
+----+---------+----+------+------------+
|   0|        0|   0|     0|           0|
+----+---------+----+------+------------+



### Ok !!! Now we have a cleaned dataset

In [17]:
# Parse the timestamp

In [18]:
from pyspark.sql.functions import udf

In [19]:
month_map = {
  'Jan': 1, 'Feb': 2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7,
  'Aug':8,  'Sep': 9, 'Oct':10, 'Nov': 11, 'Dec': 12
}

def parse_clf_time(s):
    """ Convert Common Log time format into a Python datetime object
    Args:
        s (str): date and time in Apache time format [dd/mmm/yyyy:hh:mm:ss (+/-)zzzz]
    Returns:
        a string suitable for passing to CAST('timestamp')
    """
    # NOTE: We're ignoring time zone here. In a production application, you'd want to handle that.
    return "{0:04d}-{1:02d}-{2:02d} {3:02d}:{4:02d}:{5:02d}".format(
      int(s[7:11]),
      month_map[s[3:6]],
      int(s[0:2]),
      int(s[12:14]),
      int(s[15:17]),
      int(s[18:20])
    )

u_parse_time = udf(parse_clf_time)



In [22]:
logs_df = cleaned_df.select('*', u_parse_time(split_df['timestamp']).cast('timestamp').alias('time')).drop('timestamp')


In [24]:
logs_df.head()

Py4JJavaError: An error occurred while calling o287.collectToPython.
: java.lang.IllegalArgumentException
	at org.apache.xbean.asm5.ClassReader.<init>(Unknown Source)
	at org.apache.xbean.asm5.ClassReader.<init>(Unknown Source)
	at org.apache.xbean.asm5.ClassReader.<init>(Unknown Source)
	at org.apache.spark.util.ClosureCleaner$.getClassReader(ClosureCleaner.scala:46)
	at org.apache.spark.util.FieldAccessFinder$$anon$3$$anonfun$visitMethodInsn$2.apply(ClosureCleaner.scala:449)
	at org.apache.spark.util.FieldAccessFinder$$anon$3$$anonfun$visitMethodInsn$2.apply(ClosureCleaner.scala:432)
	at scala.collection.TraversableLike$WithFilter$$anonfun$foreach$1.apply(TraversableLike.scala:733)
	at scala.collection.mutable.HashMap$$anon$1$$anonfun$foreach$2.apply(HashMap.scala:103)
	at scala.collection.mutable.HashMap$$anon$1$$anonfun$foreach$2.apply(HashMap.scala:103)
	at scala.collection.mutable.HashTable$class.foreachEntry(HashTable.scala:230)
	at scala.collection.mutable.HashMap.foreachEntry(

In [38]:
logs_df.printSchema()

root
 |-- host: string (nullable = true)
 |-- path: string (nullable = true)
 |-- status: integer (nullable = true)
 |-- content_size: integer (nullable = false)
 |-- time: timestamp (nullable = true)



In [39]:
display(logs_df)

DataFrame[host: string, path: string, status: int, content_size: int, time: timestamp]

# log Analysis

In [25]:
# Calculate statistics based on the content size.
logs_df.describe(['content_size'])

Py4JJavaError: An error occurred while calling o233.describe.
: java.lang.IllegalArgumentException
	at org.apache.xbean.asm5.ClassReader.<init>(Unknown Source)
	at org.apache.xbean.asm5.ClassReader.<init>(Unknown Source)
	at org.apache.xbean.asm5.ClassReader.<init>(Unknown Source)
	at org.apache.spark.util.ClosureCleaner$.getClassReader(ClosureCleaner.scala:46)
	at org.apache.spark.util.FieldAccessFinder$$anon$3$$anonfun$visitMethodInsn$2.apply(ClosureCleaner.scala:449)
	at org.apache.spark.util.FieldAccessFinder$$anon$3$$anonfun$visitMethodInsn$2.apply(ClosureCleaner.scala:432)
	at scala.collection.TraversableLike$WithFilter$$anonfun$foreach$1.apply(TraversableLike.scala:733)
	at scala.collection.mutable.HashMap$$anon$1$$anonfun$foreach$2.apply(HashMap.scala:103)
	at scala.collection.mutable.HashMap$$anon$1$$anonfun$foreach$2.apply(HashMap.scala:103)
	at scala.collection.mutable.HashTable$class.foreachEntry(HashTable.scala:230)
	at scala.collection.mutable.HashMap.foreachEntry(HashMap

In [26]:
file_path = 'log_cleaned'
logs_df.write.format("csv").save(file_path)

Py4JJavaError: An error occurred while calling o412.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:224)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:154)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.sc